# Model Building
modelName, balance ratio (r), utilities for "yes", "maybe", "no" as parameter input  

Assignment result outputs to excel file  

Detailed assignment quality measure outputs to print (how well the assignment achieves for workload balance, matching preference)  

Overall assignment quality measure returns as list (balance_measure defiend as workload std; preference_measure defined as propotion of "yes")


In [16]:
from gurobipy import *
import numpy as np

def IP(r,yes,maybe,no,modelName):
    
    return_result=[] 

    #-------.dat-----------
    # there are 71 papers and 21 referees
    noPapers = 71
    noReferees = 21

    # initialize the utility data
    u=[]
    import csv
    with open('paper_preferences.csv', 'rt') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in reader:
            u_temp=[]
            for j in row:
                utility=0
                if j =="yes":
                    utility=yes
                elif j=="maybe":
                    utility=maybe
                elif j=="no":
                    utility=no
                elif j=="conflict":
                    utility=0
                else:
                    utility="attribute name"
                u_temp.append(utility)
            u.append(u_temp)
    # print(u[71])
    # print(u[1][1])
    # print(u)


    #-------.mod-----------

    # create a new model
    myModel = Model( "%s" %modelName )

    # create decision variables and store them in the array myVars
    myVars =[]
    for i in range (noPapers+1 ) :  #(1,noPapers+1 ) bug here??
        myVars_temp=[]
        for j in range (noReferees+1):
            myVars_temp.append(1)
            # print("(",i,j,")")
        myVars.append(myVars_temp)
    # myVars = [ [ 1 for i in range (1, noPapers+1 ) ] for j in range (1,noReferees+1) ] #### bug here??


    for i in range(1,noPapers +1):
        for j in range (1,noReferees+1):
            curVar = myModel.addVar( vtype = GRB.BINARY , name= "x"+str(i)+"_"+str(j)) # x171 hard to distringuish
            # print("(",i,j,")")
            myVars[ i ][ j ] = curVar
    # integrate decision variables into the model
    myModel.update()


    # create a linear expression for the objective
    objExpr = LinExpr()
    for i in range(1, noPapers +1):
        for j in range (1,noReferees +1):
            curVar = myVars[ i ][ j ]
            objExpr += u[ i ][ j ] * curVar
            # print(objExpr)
    myModel.setObjective( objExpr , GRB.MAXIMIZE )


    # create constraints so that each paper is assigned to 3 referr
    for i in range( 1, noPapers+1 ):
        constExpr = LinExpr()
        for j in range( 1, noReferees+1 ):
            curVar = myVars[ i ][ j ]
            constExpr += 1 * curVar
        myModel.addConstr( lhs = constExpr , sense = GRB.EQUAL , rhs = 3, \
                           name = "3_reviewers" + "referee"+str( i ) )

    # create constraints to eliminate "conflict"
    for i in range( 1, noPapers+1 ):
        constExpr = LinExpr()
        for j in range( 1, noReferees+1 ):
            curVar = myVars[ i ][ j ]
            constExpr = 1 * curVar
            myModel.addConstr( lhs = constExpr , sense = GRB.LESS_EQUAL , rhs = u[i][j], \
                           name = "no_conflict" + "paper"+str( i )+ "referee"+str( j ))


    # create constraints for load balance
    load=[]
    load.append("this is noReferee_0, no existence")
    for j in range( 1, noReferees+1):
        constExpr = LinExpr()
        for i in range( 1, noPapers+1 ):
            curVar = myVars[ i ][ j ]
            constExpr += 1 * curVar
        load.append(constExpr)
    # for j in range( 1, noReferees+1):
    #     print(j)
    #     print(load[j])
    # print(load)

    for m in range( 1,noReferees+1):
        for n in range(1,noReferees+1):
            myModel.addConstr( lhs = load[m] , sense = GRB.LESS_EQUAL , rhs = load[n]*r , \
                           name = "balance" + str(m)+"<="+str(n) +"*r")
            # print("balance:" + str(m)+"<="+str(n) +"*r")



    # integrate objective and constraints into the model
    myModel.update()


    # write the model in a file to make sure it is constructed correctly
    myModel.write( filename = "%s_model.lp" %modelName )

    # optimize the model
    myModel.optimize()
    # print( "\nOptimal Solution:" )
    allVars = myModel.getVars()

    # output result in table form
    assignment=[]
    for curVar in allVars:
    #     print ( curVar.varName + " " + str( curVar.x ) )
        if  curVar.x==1:
            assignment.append(curVar.varName.strip('x').split('_'))

    result=np.zeros((noPapers+1,noReferees+1))
    for i in range(1,noPapers+1):
        for j in range(1,noReferees+1):
            if [str(i),str(j)] in assignment:
                result[i,j]=1
    # print(result[1])

    #---- solution output -----
    # print("-----assignment-----")
    # print(assignment)
    # for j in range(1,noReferees+1):
    #     temp=[]
    #     for i in range(1,noPapers+1):
    #         if result[i,j]==1:
    #             temp.append(i)
    #     print("Referee", j, "is assigned to paper" ,temp)

    # excel writer for assignment
    import pandas as pd
    df_result = pd.DataFrame(result)
    writer = pd.ExcelWriter("solution_%s.xlsx" %modelName, engine='xlsxwriter')
    df_result.to_excel(writer,sheet_name='%s' %modelName)
    writer.save()
    
    
    #---- model performance -----
    noYes=0
    noNo=0
    noMaybe=0
    noConflict=0
    for i in range(1,noPapers+1):
        for j in range(1,noReferees+1):
            if result[i,j]==1:
                if u[i][j]==yes:
                    noYes+=1
                elif u[i][j]==maybe:
                    noMaybe+=1
                elif u[i][j]==no:
                    noNo+=1
                elif u[i][j]==0:
                    noConflict+=1

    print("1) Assignment result in solution_%s.xlsx" %modelName)

    print("2) Measurement of assignment quality")

    print("-----noYes,noMaybe,noNo,noConflict-----")
    print(noYes,noMaybe,noNo,noConflict)
    if (noYes+noMaybe+noNo+noConflict)!=(3*71):
        print("sum error")
        return("sum error")

    print("-----Percentage of noYes,noMaybe,noNo,noConflict-----")
    print(noYes/(3*71),noMaybe/(3*71),noNo/(3*71),noConflict/(3*71))


    # # print optimal objective (no meaning here as the utility parameter is not normalized)
    # print( "-----Optimal Objective-----" )
    # print( str( myModel.ObjVal ) )

    # print workload
    print( "-----Workload Balance-----" )
    load=[]
    for j in range(1,noReferees+1):
        load_temp=0
        for i in range(1,noPapers+1):
            if result[i,j]==1:
                load_temp+=1
        load.append(load_temp)
#         print("Referee", j, "is assigned" ,load_temp,"papers")

    load_std=np.std(np.array(load))
    print("load_std=", load_std)
    
    return_result.append(load_std)
    return_result.append(noYes/(3*71))
    
    return return_result

# Numerical Experiment

Try different settings of balance ratio, utilities for "yes", "maybe", "no"  

1) Tighten constraint of workload balance by decreasing balance ratio  

2) Increase utility differences among difference preferences level 

to check feasibility and companre assignment quality


In [17]:
balance_measure=[] # workload std
preference_measure=[] # propotion of "yes"

## Parameter Setting 1
#parameter: balance ratio
r=2
#parameter: utility setting
yes=3
maybe=2
no=1

In [18]:
result=balance_measure.append(IP(2,3,2,1,"Model_1"))
result[0]
# balance_measure.append(result[0])
# preference_measure.append(result[1])

Optimize a model with 2003 rows, 1491 columns and 64113 nonzeros
Variable types: 0 continuous, 1491 integer (1491 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 319
Presolve removed 1512 rows and 66 columns
Presolve time: 0.11s
Presolved: 491 rows, 1425 columns, 58425 nonzeros
Variable types: 0 continuous, 1425 integer (1425 binary)

Root relaxation: objective 5.580000e+02, 135 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     558.0000000  558.00000  0.00%     -    0s

Explored 0 nodes (219 simplex iterations) in 0.17 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 558 319 
Pool objective bound 558

Optimal solution found (tolerance 1.00e-04)
Best o

TypeError: 'NoneType' object is not subscriptable

In [ ]:
#parameter: balance ratio
r=2

#parameter: utility setting
yes=3
maybe=2
no=1
